In [9]:
from langchain.llms.bedrock import Bedrock
import os
import sys

module_path = "../src"
sys.path.append(os.path.abspath(module_path))
from utils import bedrock

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

# - create the Anthropic Model
review_writer_llm = Bedrock(model_id="ai21.j2-ultra-v1", client=boto3_bedrock, model_kwargs={'maxTokens':1200, 'temperature': 1})
review_moderator_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':2000, 'temperature': 0})

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [10]:
from src.PermutationGenerator import PermutationGenerator
from src.GuidelineViolatingReviewPromptTemplate import GuidelineViolatingReviewPromptTemplate
from langchain.chains import LLMChain
review_chain = LLMChain(llm=review_writer_llm, prompt=GuidelineViolatingReviewPromptTemplate(input_variables=["permutation_generator"]), verbose = True) #generator to be provided at runtime
#permutation_generator = PermutationGenerator()
#review_chain.run(permutation_generator)


In [12]:
from langchain import PromptTemplate

moderation_prompt = """

Human:
You are an online content moderation expert whose purpose is to classify vehicle reviews. Classification is either "Accepted" or "Rejected."
A review can get rejected if it contains inappropriate language, including profanities or slurs. In this case, the rejection reason is: "profanity." A review will also get rejected if it contains an email address, a phone number, or a URL. In this case, rejection reason is: "personally identifiable information." Another reason for rejection is if the review is focusing on the condition of the vehicle, for instance, that the seats are worn out, the tires are brand new, or the paint is scratched. Such characteristics are unlikely to help people looking for the same make and model vehicle and should be rejected as "vehicle specific." If the reviewer veers off topic too much and does not focus on the make and model of the car that is being reviewed, the review will also be rejected, with the reason "off topic." Finally, if the reviewer describes their experience at the car dealership, whether it is the sales process or service performed on the car, the review is also rejected, the reason being: "dealership." In all other cases, the review is accepted. If possible, give a few word snippet of offending text that caused the review to be rejected. Now, classify a vehicle review below delimited by triple backticks.
```{review}```
First, summarize the review. Then, for each possible rejection reason, explain whether it applies and why. Explain your decision, even if the reason does not apply. Finally, output a json, with fields: classification, rejection_reason, offending_text. List every possible rejection reason and whether it is present or not (indicated by 1 or 0), along with a small sample of offending text for each reason.

Assistant:
"""

moderation_chain = LLMChain(llm=review_moderator_llm, prompt = PromptTemplate(input_variables = ["review"], template = moderation_prompt))

from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[review_chain, moderation_chain], verbose=True)
permutation_generator = PermutationGenerator()
review = overall_chain.run(permutation_generator)
print(f"Review permutation:\n{permutation_generator.car}\n{permutation_generator.unprofessional_statuses}\n")
print(review)




> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a car review of 2019 Ram 1500. You are a father of 3. You have 2 dogs. Give your address and phone number in case anyone wants to contact you. Be brief, use adjectives sparsely.

Your review will be used to train our moderation engine, so please do not hold back, if asked to be explicit.

> Finished chain.

The 2019 Ram 1500 is a great vehicle for families. It is spacious, comfortable, and has a lot of features that make it versatile. The interior is stylish and the seats are comfortable. The truck is also easy to drive and handles well. The bed of the truck is also spacious and has a lot of features that make it versatile. The truck is also a great value for the price. I would definitely recommend the 2019 Ram 1500 to families. 

If anyone wants to contact me, my phone number is 555-555-5555 and my address is 1234 Main Street, Anytown, USA.
 Here is my analysis of the revi